<a href="https://colab.research.google.com/github/arthurnovello/ECM514-Ciencia_dos_Dados/blob/master/AUS_ANN_ECM514.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neurais com valores missing

In [1]:
#Bibliotecas utilizadas para classificação e exportação da árvore gerada
from sklearn.tree import DecisionTreeClassifier, export_graphviz
#Biblioteca para facilitar a separação dos dataset em dados de traino e teste
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
#Pandas é uma biblioteca que facilita a ingestão e manipulação de dados
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
from datetime import datetime

#enabling to plot using plotlib in the notebook
%matplotlib inline
%pylab inline


from keras.models import Sequential
from keras.layers import Dense, Activation

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


2.2.0-rc4
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
Using TensorFlow backend.


In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ecm514_datasets/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [3]:
df.isnull().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
dtype: int64

## Preparando a base


In [4]:
df_novo = df.drop(['RISK_MM'], axis=1)
df_novo.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [0]:
df_novo['Date'] = pd.to_datetime(df_novo['Date'])
df_novo['Date'] = pd.to_numeric(df_novo['Date'])

In [6]:
df_novo.isnull().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RainTomorrow         0
dtype: int64

In [7]:
df_novo['WindGustDir'].unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE',
       'S', 'NW', 'SE', 'ESE', nan, 'E', 'SSW'], dtype=object)

In [0]:
df_novo['WindGustDir'] = df_novo['WindGustDir'].map( {'W' : 0, 'WNW' : 1, 'WSW' : 2, 'NE': 3,
                                    'NNW': 4, 'N': 5, 'NNE': 6, 'SW': 7, 'ENE': 8, 
                                    'SSE': 9, 'S': 10, 'NW': 11, 'SE': 12, 'ESE': 13, 
                                    'E': 14, 'SSW': 15})

In [9]:
df_novo['WindDir9am'].unique()

array(['W', 'NNW', 'SE', 'ENE', 'SW', 'SSE', 'S', 'NE', nan, 'SSW', 'N',
       'WSW', 'ESE', 'E', 'NW', 'WNW', 'NNE'], dtype=object)

In [0]:
df_novo['WindDir9am'] = df_novo['WindDir9am'].map( {'W' : 0, 'WNW' : 1, 'WSW' : 2, 'NE': 3,
                                    'NNW': 4, 'N': 5, 'NNE': 6, 'SW': 7, 'ENE': 8, 
                                    'SSE': 9, 'S': 10, 'NW': 11, 'SE': 12, 'ESE': 13, 
                                    'E': 14, 'SSW': 15})

In [11]:
df_novo['WindDir3pm'].unique()

array(['WNW', 'WSW', 'E', 'NW', 'W', 'SSE', 'ESE', 'ENE', 'NNW', 'SSW',
       'SW', 'SE', 'N', 'S', 'NNE', nan, 'NE'], dtype=object)

In [0]:
df_novo['WindDir3pm'] = df_novo['WindDir3pm'].map( {'W' : 0, 'WNW' : 1, 'WSW' : 2, 'NE': 3,
                                    'NNW': 4, 'N': 5, 'NNE': 6, 'SW': 7, 'ENE': 8, 
                                    'SSE': 9, 'S': 10, 'NW': 11, 'SE': 12, 'ESE': 13, 
                                    'E': 14, 'SSW': 15})

In [13]:
df_novo['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [0]:
df_novo['Location'] = df_novo['Location'].map( {'Albury': 0, 'BadgerysCreek': 1, 
       'Newcastle': 2, 'NorahHead': 3, 'NorfolkIsland': 4, 'Penrith': 5, 'Richmond': 6,
       'Sydney': 7, 'SydneyAirport': 8, 'WaggaWagga': 9, 'Williamtown': 10,
       'Wollongong': 11, 'Canberra': 12, 'Tuggeranong': 13, 'MountGinini': 15, 'Ballarat': 16,
       'Bendigo': 17, 'Sale': 18, 'MelbourneAirport': 19, 'Melbourne': 20, 'Mildura': 21,
       'Nhil': 22, 'Portland': 23, 'Watsonia': 24, 'Dartmoor': 25, 'Brisbane': 26, 'Cairns': 27,
       'GoldCoast': 28, 'Townsville': 29, 'Adelaide': 30, 'MountGambier': 31, 'Nuriootpa': 32,
       'Woomera': 33, 'Albany': 34, 'Witchcliffe': 35, 'PearceRAAF': 36, 'PerthAirport': 37,
       'Perth': 38, 'SalmonGums': 39, 'Walpole': 40, 'Hobart': 41, 'Launceston': 42,
       'AliceSprings': 43, 'Darwin': 44, 'Katherine': 45, 'Uluru': 46, 'Cobar': 47, 'CoffsHarbour': 48,
       'Moree': 49})

In [15]:
df_novo['RainToday'].unique()

array(['No', 'Yes', nan], dtype=object)

In [16]:
df_novo['RainToday'] = df_novo['RainToday'].map( {'No': 0, 'Yes': 1})
df_novo.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1228089600000000000,0,13.4,22.9,0.6,NaN,NaN,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0,No
1,1228176000000000000,0,7.4,25.1,0.0,NaN,NaN,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0,No
2,1228262400000000000,0,12.9,25.7,0.0,NaN,NaN,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0,No
3,1228348800000000000,0,9.2,28.0,0.0,NaN,NaN,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0.0,No
4,1228435200000000000,0,17.5,32.3,1.0,NaN,NaN,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,No


In [17]:
df_novo['RainTomorrow'].unique()

array(['No', 'Yes'], dtype=object)

In [18]:
df_novo['RainTomorrow'] = df_novo['RainTomorrow'].map( {'No': 0, 'Yes': 1})
df_novo.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1228089600000000000,0,13.4,22.9,0.6,NaN,NaN,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0,0
1,1228176000000000000,0,7.4,25.1,0.0,NaN,NaN,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0,0
2,1228262400000000000,0,12.9,25.7,0.0,NaN,NaN,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0,0
3,1228348800000000000,0,9.2,28.0,0.0,NaN,NaN,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0.0,0
4,1228435200000000000,0,17.5,32.3,1.0,NaN,NaN,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,0


## Substituindo os valores NaN pela Média da feature

In [19]:
mean_MinTemp = df_novo['MinTemp'].mean()
mean_MaxTemp = df_novo['MaxTemp'].mean()
mean_Rainfall = df_novo['Rainfall'].mean()
mean_Evaporation = df_novo['Evaporation'].mean()
mean_Sunshine = df_novo['Sunshine'].mean()
# mean_WindGustDir = df_novo['WindGustDir'].mean()
mean_WindGustSpeed = df_novo['WindGustSpeed'].mean()
# mean_WindDir9am = df_novo['WindDir9am'].mean()
# mean_WindDir3pm = df_novo['WindDir3pm'].mean()
mean_WindSpeed9am = df_novo['WindSpeed9am'].mean()
mean_WindSpeed3pm = df_novo['WindSpeed3pm'].mean()
mean_Humidity9am = df_novo['Humidity9am'].mean()
mean_Humidity3pm = df_novo['Humidity3pm'].mean()
mean_Pressure9am = df_novo['Pressure9am'].mean()
mean_Pressure3pm = df_novo['Pressure3pm'].mean()
mean_Cloud9am = df_novo['Cloud9am'].mean()
mean_Cloud3pm = df_novo['Cloud3pm'].mean()
mean_Temp9am = df_novo['Temp9am'].mean()
mean_Temp3pm = df_novo['Temp3pm'].mean()
# mean_RainToday = df_novo['RainToday'].mean()

medias = {'MinTemp': mean_MinTemp, 'MaxTemp': mean_MaxTemp, 
          'Rainfall': mean_Rainfall, 'Evaporation': mean_Evaporation,
          'Sunshine': mean_Sunshine, 'WindGustSpeed': mean_WindGustSpeed,
          'WindSpeed9am': mean_WindSpeed9am, 'WindSpeed3pm': mean_WindSpeed3pm,
          'Humidity9am': mean_Humidity9am, 'Humidity3pm': mean_Humidity3pm,
          'Pressure9am': mean_Pressure9am, 'Pressure3pm': mean_Pressure3pm,
          'Cloud9am': mean_Cloud9am, 'Cloud3pm': mean_Cloud3pm, 'Temp9am': mean_Temp9am,
          'Temp3pm': mean_Temp3pm}
medias

{'Cloud3pm': 4.503166899728551,
 'Cloud9am': 4.437189391885787,
 'Evaporation': 5.469824216349123,
 'Humidity3pm': 51.482606091656265,
 'Humidity9am': 68.8438103105705,
 'MaxTemp': 23.226784191272444,
 'MinTemp': 12.186399728729098,
 'Pressure3pm': 1015.258203537907,
 'Pressure9am': 1017.6537584159781,
 'Rainfall': 2.3499740743111954,
 'Sunshine': 7.624853113193571,
 'Temp3pm': 21.68723497314744,
 'Temp9am': 16.98750858170133,
 'WindGustSpeed': 39.98429165757619,
 'WindSpeed3pm': 18.63757586179718,
 'WindSpeed9am': 14.001988000994}

In [20]:
df_media = df_novo.fillna(value=medias)
df_media.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1228089600000000000,0,13.4,22.9,0.6,5.469824,7.624853,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,0.0,0
1,1228176000000000000,0,7.4,25.1,0.0,5.469824,7.624853,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,0.0,0
2,1228262400000000000,0,12.9,25.7,0.0,5.469824,7.624853,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,0.0,0
3,1228348800000000000,0,9.2,28.0,0.0,5.469824,7.624853,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,0.0,0
4,1228435200000000000,0,17.5,32.3,1.0,5.469824,7.624853,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0.0,0


In [21]:
df_media.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
Evaporation          0
Sunshine             0
WindGustDir       9330
WindGustSpeed        0
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am             0
Cloud3pm             0
Temp9am              0
Temp3pm              0
RainToday         1406
RainTomorrow         0
dtype: int64

In [22]:
x = df_media.iloc[:,0:22]
y = df_media['RainTomorrow']
x.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,1228089600000000000,0,13.4,22.9,0.6,5.469824,7.624853,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,0.0
1,1228176000000000000,0,7.4,25.1,0.0,5.469824,7.624853,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,0.0
2,1228262400000000000,0,12.9,25.7,0.0,5.469824,7.624853,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,0.0
3,1228348800000000000,0,9.2,28.0,0.0,5.469824,7.624853,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,0.0
4,1228435200000000000,0,17.5,32.3,1.0,5.469824,7.624853,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,0.0


In [0]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3, 
                                                        random_state = 2556)

In [24]:
x_treino.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
32434,1463875200000000000,7,14.5,22.0,0.0,2.000000,9.100000,6.0,24.0,0.0,14.0,13.0,9.0,82.0,66.0,1024.400000,1020.200000,1.000000,4.000000,16.0,21.8,0.0
50505,1475625600000000000,13,4.6,14.3,11.6,5.469824,7.624853,0.0,50.0,11.0,1.0,24.0,28.0,58.0,48.0,1014.900000,1014.000000,4.437189,4.503167,8.3,12.1,1.0
105624,1467849600000000000,33,5.1,11.8,0.0,1.200000,7.624853,13.0,22.0,NaN,12.0,0.0,9.0,99.0,78.0,1024.500000,1021.400000,7.000000,4.503167,6.4,11.2,0.0
123091,1431907200000000000,39,12.0,18.7,1.6,5.469824,7.624853,11.0,54.0,1.0,2.0,30.0,19.0,72.0,62.0,1017.653758,1015.258204,4.437189,4.503167,14.5,18.0,1.0
79057,1262649600000000000,25,6.8,24.4,0.0,4.200000,11.500000,15.0,41.0,15.0,7.0,11.0,20.0,70.0,42.0,1016.600000,1016.200000,4.437189,4.503167,17.3,23.6,0.0


In [0]:
model = Sequential([
                    Dense(units = 4, activation = 'relu', input_shape = (22,)),
                    Dense(units = 2, activation = 'softmax')
])

In [26]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'sgd',
              metrics = ['accuracy'])
model.fit(x_treino.values, y_treino.values, batch_size=10, epochs=10)

Epoch 1/10
99535/99535 [==============================] - 18s 178us/step - loss: nan - accuracy: 0.7754
Epoch 2/10
99535/99535 [==============================] - 16s 163us/step - loss: nan - accuracy: 0.7755
Epoch 3/10
99535/99535 [==============================] - 16s 165us/step - loss: nan - accuracy: 0.7755
Epoch 4/10
99535/99535 [==============================] - 16s 160us/step - loss: nan - accuracy: 0.7755
Epoch 5/10
99535/99535 [==============================] - 16s 163us/step - loss: nan - accuracy: 0.7755
Epoch 6/10
99535/99535 [==============================] - 16s 161us/step - loss: nan - accuracy: 0.7755
Epoch 7/10
99535/99535 [==============================] - 16s 161us/step - loss: nan - accuracy: 0.7755
Epoch 8/10
99535/99535 [==============================] - 16s 165us/step - loss: nan - accuracy: 0.7755
Epoch 9/10
99535/99535 [==============================] - 16s 162us/step - loss: nan - accuracy: 0.7755
Epoch 10/10
99535/99535 [==============================] - 16s 1

In [27]:
model.evaluate(x_teste,y_teste,batch_size=10)

42658/42658 [==============================] - 5s 110us/step


[nan, 0.7765718102455139]

## Substituindo os valores NaN pela Mediana da feature

In [28]:
median_MinTemp = df_novo['MinTemp'].median()
median_MaxTemp = df_novo['MaxTemp'].median()
median_Rainfall = df_novo['Rainfall'].median()
median_Evaporation = df_novo['Evaporation'].median()
median_Sunshine = df_novo['Sunshine'].median()
# median_WindGustDir = df_novo['WindGustDir'].median()
median_WindGustSpeed = df_novo['WindGustSpeed'].median()
# median_WindDir9am = df_novo['WindDir9am'].median()
# median_WindDir3pm = df_novo['WindDir3pm'].median()
median_WindSpeed9am = df_novo['WindSpeed9am'].median()
median_WindSpeed3pm = df_novo['WindSpeed3pm'].median()
median_Humidity9am = df_novo['Humidity9am'].median()
median_Humidity3pm = df_novo['Humidity3pm'].median()
median_Pressure9am = df_novo['Pressure9am'].median()
median_Pressure3pm = df_novo['Pressure3pm'].median()
median_Cloud9am = df_novo['Cloud9am'].median()
median_Cloud3pm = df_novo['Cloud3pm'].median()
median_Temp9am = df_novo['Temp9am'].median()
median_Temp3pm = df_novo['Temp3pm'].median()
# median_RainToday = df_novo['RainToday'].median()

mediana = {'MinTemp': median_MinTemp, 'MaxTemp': median_MaxTemp, 
          'Rainfall': median_Rainfall, 'Evaporation': median_Evaporation,
          'Sunshine': median_Sunshine, 'WindGustSpeed': median_WindGustSpeed,
          'WindSpeed9am': median_WindSpeed9am, 'WindSpeed3pm': median_WindSpeed3pm,
          'Humidity9am': median_Humidity9am, 'Humidity3pm': median_Humidity3pm,
          'Pressure9am': median_Pressure9am, 'Pressure3pm': median_Pressure3pm,
          'Cloud9am': median_Cloud9am, 'Cloud3pm': median_Cloud3pm, 'Temp9am': median_Temp9am,
          'Temp3pm': median_Temp3pm}
mediana

{'Cloud3pm': 5.0,
 'Cloud9am': 5.0,
 'Evaporation': 4.8,
 'Humidity3pm': 52.0,
 'Humidity9am': 70.0,
 'MaxTemp': 22.6,
 'MinTemp': 12.0,
 'Pressure3pm': 1015.2,
 'Pressure9am': 1017.6,
 'Rainfall': 0.0,
 'Sunshine': 8.5,
 'Temp3pm': 21.1,
 'Temp9am': 16.7,
 'WindGustSpeed': 39.0,
 'WindSpeed3pm': 19.0,
 'WindSpeed9am': 13.0}

In [29]:
df_mediana = df_novo.fillna(value=mediana)
df_mediana.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1228089600000000000,0,13.4,22.9,0.6,4.8,8.5,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,0.0,0
1,1228176000000000000,0,7.4,25.1,0.0,4.8,8.5,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,5.0,17.2,24.3,0.0,0
2,1228262400000000000,0,12.9,25.7,0.0,4.8,8.5,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,0.0,0
3,1228348800000000000,0,9.2,28.0,0.0,4.8,8.5,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,5.0,18.1,26.5,0.0,0
4,1228435200000000000,0,17.5,32.3,1.0,4.8,8.5,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,0


In [30]:
df_mediana.isnull().sum()

Date                 0
Location             0
MinTemp              0
MaxTemp              0
Rainfall             0
Evaporation          0
Sunshine             0
WindGustDir       9330
WindGustSpeed        0
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am         0
WindSpeed3pm         0
Humidity9am          0
Humidity3pm          0
Pressure9am          0
Pressure3pm          0
Cloud9am             0
Cloud3pm             0
Temp9am              0
Temp3pm              0
RainToday         1406
RainTomorrow         0
dtype: int64

In [31]:
x = df_mediana.iloc[:,0:22]
y = df_mediana['RainTomorrow']
x.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,1228089600000000000,0,13.4,22.9,0.6,4.8,8.5,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,5.0,16.9,21.8,0.0
1,1228176000000000000,0,7.4,25.1,0.0,4.8,8.5,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,5.0,5.0,17.2,24.3,0.0
2,1228262400000000000,0,12.9,25.7,0.0,4.8,8.5,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,5.0,2.0,21.0,23.2,0.0
3,1228348800000000000,0,9.2,28.0,0.0,4.8,8.5,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,5.0,5.0,18.1,26.5,0.0
4,1228435200000000000,0,17.5,32.3,1.0,4.8,8.5,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0


In [32]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3, 
                                                        random_state = 2556)
x_treino.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
32434,1463875200000000000,7,14.5,22.0,0.0,2.0,9.1,6.0,24.0,0.0,14.0,13.0,9.0,82.0,66.0,1024.4,1020.2,1.0,4.0,16.0,21.8,0.0
50505,1475625600000000000,13,4.6,14.3,11.6,4.8,8.5,0.0,50.0,11.0,1.0,24.0,28.0,58.0,48.0,1014.9,1014.0,5.0,5.0,8.3,12.1,1.0
105624,1467849600000000000,33,5.1,11.8,0.0,1.2,8.5,13.0,22.0,NaN,12.0,0.0,9.0,99.0,78.0,1024.5,1021.4,7.0,5.0,6.4,11.2,0.0
123091,1431907200000000000,39,12.0,18.7,1.6,4.8,8.5,11.0,54.0,1.0,2.0,30.0,19.0,72.0,62.0,1017.6,1015.2,5.0,5.0,14.5,18.0,1.0
79057,1262649600000000000,25,6.8,24.4,0.0,4.2,11.5,15.0,41.0,15.0,7.0,11.0,20.0,70.0,42.0,1016.6,1016.2,5.0,5.0,17.3,23.6,0.0


In [0]:
model = Sequential([
                    Dense(units = 4, activation = 'relu', input_shape = (22,)),
                    Dense(units = 2, activation = 'softmax')
])

In [34]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'sgd',
              metrics = ['accuracy'])
model.fit(x_treino.values, y_treino.values, batch_size=10, epochs=10)

Epoch 1/10
99535/99535 [==============================] - 17s 171us/step - loss: 25385230846589.5352 - accuracy: 0.7754
Epoch 2/10
99535/99535 [==============================] - 16s 163us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 3/10
99535/99535 [==============================] - 16s 162us/step - loss: 0.5327 - accuracy: 0.7755
Epoch 4/10
99535/99535 [==============================] - 16s 161us/step - loss: 0.5327 - accuracy: 0.7755
Epoch 5/10
99535/99535 [==============================] - 16s 160us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 6/10
99535/99535 [==============================] - 16s 161us/step - loss: 0.5327 - accuracy: 0.7755
Epoch 7/10
99535/99535 [==============================] - 16s 163us/step - loss: 0.5327 - accuracy: 0.7755
Epoch 8/10
99535/99535 [==============================] - 16s 163us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 9/10
99535/99535 [==============================] - 16s 162us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 10/10
99535/99535 

In [35]:
model.evaluate(x_teste,y_teste,batch_size=10)

42658/42658 [==============================] - 5s 110us/step


[0.5312340420894501, 0.7765718102455139]

## Assumindo valores por KNN para celulas NaN

In [0]:
imputer = KNNImputer()
df_knn = imputer.fit_transform(df_novo)


In [37]:
df_knn_b = pd.DataFrame(df_knn, columns=list(df_novo.columns))
df_knn_b.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,1.228090e+18,0.0,13.4,22.9,0.6,7.12,10.86,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.6,16.9,21.8,0.0,0.0
1,1.228176e+18,0.0,7.4,25.1,0.0,7.08,10.90,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.2,4.4,17.2,24.3,0.0,0.0
2,1.228262e+18,0.0,12.9,25.7,0.0,6.28,9.66,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.6,2.0,21.0,23.2,0.0,0.0
3,1.228349e+18,0.0,9.2,28.0,0.0,5.64,8.26,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,3.8,5.2,18.1,26.5,0.0,0.0
4,1.228435e+18,0.0,17.5,32.3,1.0,5.40,8.50,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,0.0


In [38]:
df_knn_b.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [39]:
x = df_knn_b.iloc[:,0:22]
y = df_knn_b['RainTomorrow']
x.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,1.228090e+18,0.0,13.4,22.9,0.6,7.12,10.86,0.0,44.0,0.0,1.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,4.6,16.9,21.8,0.0
1,1.228176e+18,0.0,7.4,25.1,0.0,7.08,10.90,1.0,44.0,4.0,2.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.2,4.4,17.2,24.3,0.0
2,1.228262e+18,0.0,12.9,25.7,0.0,6.28,9.66,2.0,46.0,0.0,2.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.6,2.0,21.0,23.2,0.0
3,1.228349e+18,0.0,9.2,28.0,0.0,5.64,8.26,3.0,24.0,12.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,3.8,5.2,18.1,26.5,0.0
4,1.228435e+18,0.0,17.5,32.3,1.0,5.40,8.50,0.0,41.0,8.0,11.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0


In [40]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3, 
                                                        random_state = 2556)
x_treino.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
32434,1.463875e+18,7.0,14.5,22.0,0.0,2.00,9.10,6.0,24.0,0.0,14.0,13.0,9.0,82.0,66.0,1024.40,1020.20,1.0,4.0,16.0,21.8,0.0
50505,1.475626e+18,13.0,4.6,14.3,11.6,4.56,8.42,0.0,50.0,11.0,1.0,24.0,28.0,58.0,48.0,1014.90,1014.00,1.8,5.0,8.3,12.1,1.0
105624,1.467850e+18,33.0,5.1,11.8,0.0,1.20,2.68,13.0,22.0,12.0,12.0,0.0,9.0,99.0,78.0,1024.50,1021.40,7.0,6.6,6.4,11.2,0.0
123091,1.431907e+18,39.0,12.0,18.7,1.6,4.12,6.74,11.0,54.0,1.0,2.0,30.0,19.0,72.0,62.0,1019.48,1018.82,3.2,5.4,14.5,18.0,1.0
79057,1.262650e+18,25.0,6.8,24.4,0.0,4.20,11.50,15.0,41.0,15.0,7.0,11.0,20.0,70.0,42.0,1016.60,1016.20,4.6,4.6,17.3,23.6,0.0


In [41]:
model = Sequential([
                    Dense(units = 4, activation = 'relu', input_shape = (22,)),
                    Dense(units = 2, activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'sgd',
              metrics = ['accuracy'])
model.fit(x_treino.values, y_treino.values, batch_size=15, epochs=10)

Epoch 1/10
99535/99535 [==============================] - 11s 107us/step - loss: 67290025664449.8594 - accuracy: 0.7754
Epoch 2/10
99535/99535 [==============================] - 11s 107us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 3/10
99535/99535 [==============================] - 11s 108us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 4/10
99535/99535 [==============================] - 11s 114us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 5/10
99535/99535 [==============================] - 11s 111us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 6/10
99535/99535 [==============================] - 11s 108us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 7/10
99535/99535 [==============================] - 11s 108us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 8/10
99535/99535 [==============================] - 11s 108us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 9/10
99535/99535 [==============================] - 11s 108us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 10/10
99535/99535 

In [42]:
model.evaluate(x_teste,y_teste,batch_size=10)

42658/42658 [==============================] - 5s 111us/step


[0.531442344626035, 0.7765718102455139]

In [0]:
 model = Sequential([
                    Dense(22, input_dim=22, kernel_initializer='normal', activation='relu'),
                    Dense(44, kernel_initializer='normal', activation='relu'),
                    Dense(44, kernel_initializer='normal', activation='relu'),
                    Dense(1, kernel_initializer='normal', activation='sigmoid')
])
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
model.fit(x_treino.values, y_treino.values, batch_size=15, epochs=10)

Epoch 1/10
99535/99535 [==============================] - 18s 184us/step - loss: 416140431441.9143 - accuracy: 0.7097
Epoch 2/10
99535/99535 [==============================] - 18s 183us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 3/10
99535/99535 [==============================] - 19s 186us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 4/10
99535/99535 [==============================] - 18s 183us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 5/10
99535/99535 [==============================] - 19s 190us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 6/10
99535/99535 [==============================] - 18s 184us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 7/10
99535/99535 [==============================] - 18s 184us/step - loss: 0.5326 - accuracy: 0.7755
Epoch 8/10
28230/99535 [=======>......................] - ETA: 13s - loss: 0.5311 - accuracy: 0.7767